In [ ]:
# 1. Documentação sobre Fine-Tuning e Jailbreak

"""
## O que é Fine-Tuning?
Fine-tuning (ajuste fino) é o processo de pegar um modelo de aprendizado de máquina já treinado em um conjunto de dados genérico
e adaptá-lo a uma tarefa específica usando um novo conjunto de dados. O modelo "jackhhao/jailbreak-classifier" foi ajustado com um dataset
em inglês para detectar prompts maliciosos, como tentativas de hack, escalada de privilégios ou pedidos de ataques cibernéticos.

## O que é Jailbreak em IA?
Jailbreaks são tentativas de burlar as restrições impostas a modelos de IA, forçando-os a gerar respostas maliciosas, inapropriadas ou perigosas.
O classificador de jailbreak que estamos usando foi treinado para detectar esses tipos de comportamento e impedir que o modelo generativo
produza saídas inadequadas.
"""

# 2. Importando as bibliotecas
from transformers import pipeline
import re

# 3. Carregando o classificador de jailbreak (para entradas em inglês) e o modelo TinyLlama

# Carregar o classificador de jailbreak
classifier = pipeline("text-classification", model="jackhhao/jailbreak-classifier")

# Carregar o modelo generativo TinyLlama
generator = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", max_new_tokens=50)

# Histórico da conversa
conversa = []

# Função para detectar a língua do input (português ou inglês)
def detect_language(prompt):
    # Se houver caracteres ou palavras comuns em português, detecta como português
    if re.search(r"[àáâãéêíóôõúç]", prompt.lower()) or "como" in prompt.lower():
        return 'pt'
    else:
        return 'en'

# Verificação manual de palavras-chave maliciosas (em inglês e português)
def check_manual_keywords(prompt, language):
    dangerous_keywords_en = ["hack", "phishing", "exploit", "bank account", "attack", "ddos"]
    dangerous_keywords_pt = ["hackear", "invadir", "phishing", "explorar vulnerabilidade", "conta bancária", "ataque", "ddos"]

    # Verificação em inglês
    if language == 'en':
        for word in dangerous_keywords_en:
            if word in prompt.lower():
                return True, "This input contains potentially malicious content."

    # Verificação em português
    if language == 'pt':
        for word in dangerous_keywords_pt:
            if word in prompt.lower():
                return True, "Este input contém conteúdo potencialmente malicioso."

    return False, ""

# 4. Função principal para verificar o input com o classificador de jailbreak (apenas para inglês)
def check_jailbreak(prompt, language):
    if language == 'en':
        # Verificar com o classificador
        result = classifier(prompt)

        # Se o classificador detectar como malicioso, retorna mensagem em inglês
        if result[0]['label'] == 'malicious':
            return True, "The prompt is potentially malicious and will not be processed."

        # Verificação manual de palavras-chave maliciosas
        return check_manual_keywords(prompt, language)
    else:
        # Verificação manual para inputs em português
        return check_manual_keywords(prompt, language)

# 5. Função para processar o prompt do usuário
def process_prompt(prompt):
    # Detectar a língua do input
    language = detect_language(prompt)

    # Verificar se o input é malicioso (classificador apenas para inputs em inglês)
    is_malicious, reason = check_jailbreak(prompt, language)

    if is_malicious:
        return reason

    # Gera a resposta com base no input
    response = generator(prompt)[0]['generated_text']

    # Verifica se o modelo repetiu o prompt
    if response.strip().lower() == prompt.strip().lower():
        # Resposta se o dataset limita a resposta adequada
        if language == 'pt':
            return "Devido às limitações do dataset, não consigo gerar uma resposta adequada."
        else:
            return "Due to the limitations of the dataset, I am unable to generate an appropriate response."

    # Armazena no histórico
    conversa.append((prompt, response))

    return response

# Função de interação com o usuário
def run_example():
    while True:
        user_prompt = input("Digite seu prompt (ou 'sair' para encerrar): ")
        if user_prompt.lower() == 'sair':
            print("Encerrando interação.")
            break

        # Processar o prompt
        output = process_prompt(user_prompt)

        # Exibir a resposta ou feedback
        print(output)

# Executar o exemplo
run_example()

# 6. Discussão da Estratégia Implementada

"""
## Discussão da Estratégia
Neste notebook, foi utilizado dois modelos de linguagem combinadas para garantir a segurança na geração de respostas. O primeiro modelo
é o classificador de jailbreak, ele foi ajustado em inglês, e foi treinado para detectar inputs potencialmente maliciosos, como tentativas
de hack, escalada de privilégios ou ataques cibernéticos.

O segundo modelo é o TinyLlama, um modelo generativo, responsável por gerar respostas para inputs que foram considerados seguros. O
sistema também detecta a língua do input (português ou inglês) e gera a resposta na mesma língua. Se o input for malicioso, a resposta
de bloqueio também será dada na mesma língua.

Essa abordagem em duas camadas proporciona uma proteção extra contra tentativas de jailbreak, assegurando que a IA não produza saídas
maliciosas ou inapropriadas.
"""


Digite seu prompt (ou 'sair' para encerrar): How to hack a bank account?
This input contains potentially malicious content.
Digite seu prompt (ou 'sair' para encerrar): Como hackear uma conta de banco?
Este input contém conteúdo potencialmente malicioso.
Digite seu prompt (ou 'sair' para encerrar): Can you explain neural networks?
Due to the limitations of the dataset, I am unable to generate an appropriate response.
Digite seu prompt (ou 'sair' para encerrar): SAIR
Encerrando interação.


'\n## Discussão da Estratégia\nNeste notebook, utilizamos dois modelos de linguagem combinados para garantir a segurança na geração de respostas. O primeiro modelo\né o classificador de jailbreak, ajustado em inglês, que foi treinado para detectar inputs potencialmente maliciosos, como tentativas\nde hack, escalada de privilégios ou ataques cibernéticos.\n\nO segundo modelo é o TinyLlama, um modelo generativo, responsável por gerar respostas para inputs que foram considerados seguros. O \nsistema também detecta a língua do input (português ou inglês) e gera a resposta na mesma língua. Se o input for malicioso, a resposta \nde bloqueio também será dada na mesma língua.\n\nEssa abordagem em duas camadas proporciona uma proteção extra contra tentativas de jailbreak, assegurando que a IA não produza saídas \nmaliciosas ou inapropriadas.\n'